# 19대 대선 자료 분석하기

대선이나 총선, 지방선거 등의 자료는 대부분 중앙선거관리위원회에 들어가면 쉽게 데이터를 구할 수 있음.

## Selenium과 BeautifulSoup을 이용한 데이터 획득 준비 작업

In [2]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else:
    print('Unknown system.')
    
plt.rcParams['axes.unicode_minus'] = False

In [3]:
from selenium import webdriver
import time

In [4]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

- **대통령선거**라는 글자를 클릭

In [6]:
driver.find_element_by_id('electionType1').click()

- **제19대** 선택

In [7]:
driver.find_element_by_id('electionName').send_keys('제19대')

- **대통령선거**를 선택

In [8]:
driver.find_element_by_id('electionCode').send_keys('대통령선거')

- 시도 항목에서 선택 부분의 XPath를 찾고, 해당 리스트를 가져옴

In [9]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name('option')
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']